In [1]:
%%capture
# RUN
# This will install all the requirements inside of `requirements.txt` into your current working environment. 
%pip install -r requirements.txt

In [2]:
# RUN
import numpy as np
import pandas as pd
import re
import selenium
import math
import threading
import asyncio
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, NoSuchWindowException, StaleElementReferenceException
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By


In [3]:
#Function that webscrapes through links in table and adds if anything valid is found
#Input: Dataframe with twitch link
#Output: void
   
def element_search(pattern, link):                
    match = re.search(pattern, link)
    if match:
        if pattern == "youtube":
            if re.search("channel", link): 
#                 print("MATCH: " + link)
                return link
            else:
                return None
        else:
#             print("MATCH: " + link)
            return link
    else: 
#         print("NO MATCH: " + link)
        return None 
    

# async def get_link(element):
#     val = await element.get_attribute("href")
#     return val

def find(driver):
    element = driver.find_elements_by_xpath("//a[@href]")
    if element:
        return element
    else:
        return False
    

async def get_link(element):
    return element.get_attribute("href")

async def main(element):
    val = await get_link(element)
    return val

def webscraper(twitchdf):
    driver = webdriver.Chrome('/Users/yyamada/downloads/chromedriver')
    twitch_names = twitchdf["DisplayName"]
    twitch_links = twitchdf["About Pages"]
    index = 0
    noneCount = 0
    for link in twitch_links:
        index = twitchdf[twitchdf['About Pages']== link].index.values[0]
        name = twitch_names.loc[index]
        final_info = [name, link, None, None, None, None]
        driver.get(link)
        elements = WebDriverWait(driver, 5).until(find)
        email = None
        twitter = None
        discord = None
        youtube = None
        
        for element in elements:
            try: 
                href = asyncio.run(main(element))
                if discord == None: 
                    discord = element_search("discord", href)
                if email == None :
                    email = element_search("gmail.com", href)
                if twitter == None: 
                    twitter = element_search("twitter", href)
                if youtube == None: 
                    youtube = element_search("youtube", href)
            except StaleElementReferenceException:
                print("Errored")
                pass 
        
        final_info[2] = email
        final_info[3] = twitter
        final_info[4] = discord
        final_info[5] = youtube
        
        if final_info == [name, link, None, None, None, None]: 
            noneCount += 1
        
        final_df.loc[len(final_df.index)] = final_info 
        print(noneCount)



In [4]:
# RUN
# table we'll be working with, add /about to twitchlinks in the beginning 
def load_data(destination_file):
    table = pd.read_csv(destination_file)
    twitch_links = table['Twitch Link'] 
    about_pages = []
    for link in twitch_links:
        try:
            about_pages.append(link + "/about")
        except TypeError:
            # Handle Error 
            about_pages.append(link)
            pass 
    table['About Pages'] = about_pages 
    return table 

def divide_df(df, n): 
    divided_df = []
    length = len(df)
    interval = math.floor(length / n)
    for i in range(n): 
        if (i == (n-1)):
            divided_df.append(df.loc[interval*i:])
        else:
            divided_df.append(df.loc[interval * i : interval * (i + 1) - 1])
    return divided_df

In [5]:
# Python threading 
def parse_threading(df_list):
    threads = []
    for df in df_list:
        threads.append(threading.Thread(target=webscraper, args=(df,)))
    for thread in threads:
        thread.start()
    
    for thread in threads:
        thread.join()
    
    print("I'm done!")


In [283]:
# all the functions called. This will output everything 
new_table = load_data('scrub_list_example.csv')
divided_dfs = divide_df(new_table, 4)
final_df = pd.DataFrame({'DisplayName': [], 'Twitch Link': [], 'Email': [], 'Twitter': [], 'Discord': [],'YouTube': []})
parse_threading(divided_dfs)


final_df = final_df.drop_duplicates()
pd.set_option('display.max_rows', None)  # or 1000
display(final_df)
# BUT: //FIXME  some valid links like "youtube/channelname" is not being caught, which is probably because 
# the StateElement error is happening and the link isn't being parsed. We need to figure that out. 
# and then we need to figure out:
# 1. Make the whole thing faster, right now its like 30links/min, to parse through 600, that'll take too much time

/var/folders/8h/zcvnk3m157q9bs_yhf9lxx5w0000gn/T/ipykernel_70589/4186244130.py:42: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/Users/yyamada/downloads/chromedriver')
/var/folders/8h/zcvnk3m157q9bs_yhf9lxx5w0000gn/T/ipykernel_70589/4248685528.py:25: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  element = driver.find_elements_by_xpath("//a[@href]")


Errored
Errored
Errored
0
Errored
0


/var/folders/8h/zcvnk3m157q9bs_yhf9lxx5w0000gn/T/ipykernel_70589/4248685528.py:25: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  element = driver.find_elements_by_xpath("//a[@href]")


0
0
Errored
Errored
Errored
0


/var/folders/8h/zcvnk3m157q9bs_yhf9lxx5w0000gn/T/ipykernel_70589/4248685528.py:25: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  element = driver.find_elements_by_xpath("//a[@href]")


...

/var/folders/8h/zcvnk3m157q9bs_yhf9lxx5w0000gn/T/ipykernel_70589/4248685528.py:25: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  element = driver.find_elements_by_xpath("//a[@href]")


16


/var/folders/8h/zcvnk3m157q9bs_yhf9lxx5w0000gn/T/ipykernel_70589/4248685528.py:25: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  element = driver.find_elements_by_xpath("//a[@href]")


17
I'm done!


,DisplayName,Twitch Link,Email,Twitter,Discord,YouTube
0,AdamRedmondPlays,https://twitch.tv/AdamRedmondPlays/about,None,https://twitter.com/ItsAdamRedmond,None,None
1,AshlieCeleste,https://twitch.tv/AshlieCeleste/about,None,https://twitter.com/StudioVulcan,None,None
2,makelchs,https://twitch.tv/makelchs/about,None,None,None,https://www.youtube.com/channel/UCCs1YZHxUTIbL...
3,AestheticPowerTV,https://twitch.tv/AestheticPowerTV/about,None,None,https://discord.gg/6NQdzD7,https://www.youtube.com/channel/UCkMDfYRCj9C1U...
4,AdamTheEnginerd,https://twitch.tv/AdamTheEnginerd/about,None,None,None,https://www.youtube.com/channel/UC7lDfLUgSw4Ji...
5,ASMRPunk,https://twitch.tv/ASMRPunk/about,None,None,https://discord.gg/PkbvCvM,https://www.youtube.com/channel/UCQrBcp14UDA7U...
6,blwriter,https://twitch.tv/blwriter/about,None,https://twitter.com/BlackWriter_VR,https://discord.com/invite/UJTtNHu,https://www.youtube.com/channel/UCvp1XZk7Jwtwr...
7,sowhan,https://twitch.tv/sowhan/about,None,None,None,https://www.youtube.com/channel/UC7iIGOn477JNY...
8,Adamyid,https://twitch.tv/Adamyid/about,None,https://twitter.com/adamyid,https://discord.gg/d5EZdTp,None
9,asmrshanny,https://twitch.tv/asmrshanny/about,None,https://www.twitter.com/asmrshanny,None,None


In [6]:
def counter(df, categories): 
    for category in categories: 
        count = 0
        category_links = df[category]
        for link in category_links:
            if link != None: 
                count += 1
        print(category + ": " + str((count / len(df.index)) *100) + "%")
    return

In [285]:
counter(final_df, ["Email", "Twitter", "Discord", "YouTube"])

Email: 1.0714285714285714%
Twitter: 65.0%
Discord: 50.357142857142854%
YouTube: 39.464285714285715%


In [237]:
dub = "discord"
sin = 'discord' 
pattern string 
if re.search(sin, "discordlala"): 
    print("yes")

yes
